In [1]:
from tqdm import tqdm
import simplejson as json
import requests
from pyquery import PyQuery as pq
from PIL import Image, ImageOps, ImageEnhance    # Pillow

import os
import urllib.parse
from glob import glob

In [13]:
autocontrastcutoff = 25   # Cut off how many percent of histogram?
imagesharpness = 1.8   # How much to sharpen?  Was 1.5

In [3]:
urlswanted = [
    "https://essss.library.vanderbilt.edu/islandora/object/essss%3A38558#page/1/mode/2up"
]

In [6]:
urlpre = "https://essss.library.vanderbilt.edu/iiif/2/"
urlpost = "/full/pct:100/0/default.jpg"

In [7]:
headers = {
    "Accept": "image/avif,image/webp,image/apng,image/*,*/*;q=0.8",
    "Accept-Encoding": "gzip, deflate, br",
    "Accept-Language": "en-US,en;q=0.9",
    "Connection": "keep-alive",
    "Cookie": "has_js=1; _ga=GA1.2.1752803547.1599356991; _gid=GA1.2.880756344.1599356991; _gat=1",
    "Host": "essss.library.vanderbilt.edu",
    "Referer": "https://essss.library.vanderbilt.edu/islandora/object/essss%3A38558",
    "Sec-Fetch-Dest": "image",
    "Sec-Fetch-Mode": "no-cors",
    "Sec-Fetch-Site": "same-origin"
}

In [8]:
for startpage in urlswanted:
    r = requests.get(startpage)
    html = r.content
    bookname = pq(html)("li.active")[0].text
    bookdir = "books/" + bookname + "/"
    rawdir = bookdir + "raw/"
    processeddir = bookdir + "processed/"
    for mydir in [rawdir, processeddir]:
        os.makedirs(mydir, exist_ok=True)
    print(f"Processing {bookname}")
    rawjson = r.text.split(',"pageCount"')[0].split(',"pages":')[1]
    rawlist = json.loads(rawjson)
    for mypage in tqdm(rawlist[:]):
        targetpage = mypage['page'].zfill(3)
        targetfilename = rawdir + targetpage + ".jpg"
        targeturl = urlpre + urllib.parse.unquote(mypage['identifier']) + urlpost
        if not os.path.exists(targetfilename):
            r = requests.get(targeturl, headers=headers)
            if r.status_code == 200:
                with open(targetfilename, "wb") as outfile:
                    outfile.write(r.content)

100%|█████████████████████████████████████████████████████████████████████████████| 330/330 [00:00<00:00, 12228.94it/s]

Processing Livro de batismos de livres, 1774-1787


In [14]:
for bookdir in tqdm(list(glob("books/*"))):
    print(f"Processing {bookdir}")
    rawdir = bookdir + "/raw/"
    processeddir = bookdir + "/processed/"
    rawfiles = glob(rawdir + "*.*")
    for rawfile in tqdm(rawfiles):
        rawfile = rawfile.replace("\\", "/")
        targetfile = rawfile.replace("/raw/", "/processed/")
        if not os.path.exists(targetfile):
            im = Image.open(rawfile)
            if im.width > im.height:
                im = im.rotate(90)
            im = ImageOps.autocontrast(im, cutoff=autocontrastcutoff)
            enhancer = ImageEnhance.Sharpness(im)
            im = enhancer.enhance(imagesharpness)
            im.save(targetfile)


  0%|                                                                                          | 0/330 [00:00<?, ?it/s]

Processing books\Livro de batismos de livres, 1774-1787



 20%|████████████████▏                                                                | 66/330 [00:34<02:23,  1.85it/s]


 40%|████████████████████████████████                                                | 132/330 [01:06<01:35,  2.08it/s]


 60%|████████████████████████████████████████████████                                | 198/330 [01:39<01:10,  1.86it/s]


 80%|████████████████████████████████████████████████████████████████                | 264/330 [02:10<00:30,  2.14it/s]


100%|████████████████████████████████████████████████████████████████████████████████| 330/330 [02:40<00:00,  2.05it/s]


100%|███████████████████████████████████████████████████████████████████████████████████| 1/1 [02:40<00:00, 160.98s/it]


In [ ]:
rawfiles